##### Imports

In [1]:
from asvFormula.bayesianNetworks.bayesianNetwork import *
from asvFormula.bayesianNetworks import networkSamplesPath
from asvFormula.datasetManipulation import *
from asvFormula.exactASV import showASVandShapleyFor, ASV
from asvFormula.classesSizes.digraph import hasUnderlyingTree
import random 
from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination
import numpy as np

#TODO: I should add classes and assign responsibilities to them. All of the functions are mixed and I have multiple variables flying. 
#TODO: Fix the seed and the random so that I can reproduce the results. Right now it's not changing anything the seed method. 

In [ ]:
import importlib
import asvFormula

# Reload the module
importlib.reload(asvFormula.classesSizes.digraph)

### Running ASV for Cancer bayesian network

#### Train the decision tree model

In [ ]:
cancerNetworkPath = networkSamplesPath + "/cancer.bif"
BNmodel = BIFReader(cancerNetworkPath).get_model()
variableToPredict = "Pollution"
numberOfSamples = 2000
treeMaxDepth = 3

BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeData(BNmodel, variableToPredict, numberOfSamples, treeMaxDepth)

#### Compute the ASV vs the Shapley Value

In [ ]:
asvCalc = ASV(BNmodel, dtTreeClassifier, BNInference, valuesPerFeature)

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

#showASVandShapleyFor(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc)


#### Mean prediction of model

In [10]:
# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(cancerNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)
showMeanPredictionOfModel('high', variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

In [ ]:
showMeanPredictionOfModel('high', variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

### Running ASV for Child bayesian network

In [2]:
childNetworkPath = networkSamplesPath + "/child.bif"

def childBNAsTree(childNetworkPath):
    treeBNChild = BIFReader(childNetworkPath).get_model()

    #I remove this edges so that it is a tree and we can work with it
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungParench', 'Grunting')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungParench', 'HypoxiaInO2')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'HypoxiaInO2', 'LowerBodyO2')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'CardiacMixing', 'HypDistrib')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'Sick', 'Age')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungFlow', 'ChestXray')

    assert hasUnderlyingTree(treeBNChild)
    return treeBNChild


treeBNChild = childBNAsTree(childNetworkPath)
variableToPredict = "Age"
numberOfSamples = 100
treeMaxDepth = 3

BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeData(treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth)


  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.85


### ASV + Shapley

In [ ]:
asvCalc = ASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature)

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

showASVandShapleyFor(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc, progress=True)

### Mean Prediction

In [3]:
# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(childNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)

In [6]:
showMeanPredictionOfModel('0-3_days', variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

INFO:pgmpy:Using Age state as number instead of name.
INFO:pgmpy:Using Age state as number instead of name.
INFO:pgmpy:Using Age state as number instead of name.
INFO:pgmpy:Using Age state as number instead of name.
INFO:pgmpy:Using Age state as number instead of name.
INFO:pgmpy:Using Age state as number instead of name.
INFO:pgmpy:Using Age state as number instead of name.


Predicting the value 0-3_days for the variable Age
Mean prediction value for the model: 1.09
Estimated value for shap explainer: 0.725
Probability in the bayesian network: 0.6489918355500001


In [13]:
import importlib
import asvFormula
import asvFormula.bayesianNetworks
import asvFormula.bayesianNetworks.bayesianNetwork

# Reload the module
importlib.reload(asvFormula.bayesianNetworks.bayesianNetwork)

<module 'asvFormula.bayesianNetworks.bayesianNetwork' from '/home/echu/tesis/pasantia-BICC/asvFormula/bayesianNetworks/bayesianNetwork.py'>